In [1]:
import os

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

In [16]:
from pyspark.sql.functions import to_json

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.mongodb.spark:mongo-spark-connector_2.11:2.2.0 pyspark-shell'

In [3]:
conf = SparkConf()\
    .setAppName("pyspark mongo")\
    .setMaster("local[*]")\
    .set("spark.mongodb.input.uri", "mongodb://localhost:27017/")\
    .set("spark.mongodb.input.database", "sherpa")\
    .set("spark.mongodb.input.collection", "reports")

In [4]:
sc = SparkContext(conf=conf)

In [5]:
sql_context = SQLContext(sc)

In [6]:
pipeline = "{'$match': {'jobMetaData.tag': 'join_CPU_10GB_11-09-2017-1'}}"

In [7]:
df = sql_context\
    .read\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("pipeline", pipeline)\
    .load()

In [8]:
df.count()

51

In [26]:
use_cols = [to_json(df[dtype[0]]) if dtype[1].startswith('struct') else df[dtype[0]] 
            for dtype in df.dtypes if dtype[0] != '_id']

In [27]:
df.select(use_cols).show(2)

+--------------------+--------+-----------+---------+--------------------+----+-----------------------+---------+------------+--------------------------+-------------+------------------+----------+-------------------+-------+------------+------------------------+-------------------------+--------------------------+--------------------------+----------------+--------------------+
|          allConfigs|attempts|clientSeqNo|clusterID| structstojson(conf)|cost|structstojson(counters)|cpuMetric|isBestConfig|structstojson(jobMetaData)|latencyMetric|      memoryMetric|originator|servedToClientCount|  state|tenzingSeqNo|throughputJobLevelMetric|throughputTaskLevelMetric|throughputTaskLevelMetric2|throughputTaskLevelMetric3|tunedParamsTypes|          workloadID|
+--------------------+--------+-----------+---------+--------------------+----+-----------------------+---------+------------+--------------------------+-------------+------------------+----------+-------------------+-------+-----------

In [17]:
df.select(to_json(df['jobMetaData'])).show(10)

+--------------------------+
|structstojson(jobMetaData)|
+--------------------------+
|      {"executionTime":...|
|      {"executionTime":...|
|      {"executionTime":...|
|      {"executionTime":...|
|      {"executionTime":...|
|      {"executionTime":...|
|      {"executionTime":...|
|      {"executionTime":...|
|      {"executionTime":...|
|      {"executionTime":...|
+--------------------------+
only showing top 10 rows



In [11]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- allConfigs: string (nullable = true)
 |-- attempts: integer (nullable = true)
 |-- clientSeqNo: integer (nullable = true)
 |-- clusterID: string (nullable = true)
 |-- conf: struct (nullable = true)
 |    |-- mapreduce_reduce_memory_mb: string (nullable = true)
 |    |-- mapreduce_map_cpu_vcores: string (nullable = true)
 |    |-- java_heap_size_map: string (nullable = true)
 |    |-- mapreduce_reduce_shuffle_input_buffer_percent: string (nullable = true)
 |    |-- mapreduce_reduce_shuffle_parallelcopies: string (nullable = true)
 |    |-- mapreduce_map_memory_mb: string (nullable = true)
 |    |-- java_heap_size_reduce: string (nullable = true)
 |    |-- mapreduce_input_fileinputformat_split_maxsize: string (nullable = true)
 |    |-- mapreduce_job_reduces: string (nullable = true)
 |    |-- mapreduce_reduce_shuffle_merge_percent: string (nullable = true)
 |    |-- mapreduce_reduce_input_buffer_percen